In [44]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import re
import time
import asyncio

In [45]:
START_YEAR = 2020
END_YEAR=2025
LOOKBACK_PERIOD = range(START_YEAR, END_YEAR+1)
LOOKBACK_PERIOD=list(LOOKBACK_PERIOD)
if 2020 in LOOKBACK_PERIOD:
    LOOKBACK_PERIOD.remove(2020)
print(LOOKBACK_PERIOD)

[2021, 2022, 2023, 2024, 2025]


In [46]:
base = "https://www.teamrankings.com/ncaa-basketball/"
features = {"Offensive Efficiency":"stat/offensive-efficiency",
#             "Floor %":"stat/floor-percentage", 
#             "Shooting %":"stat/shooting-pct",
            "Effective Field Goal %":"stat/effective-field-goal-pct",
            "Three Point %":"stat/three-point-pct",
            "Two Point %":"stat/two-point-pct",
            "Free Throw %":"stat/free-throw-pct",
            "FT Attempted per FG Attempted": "stat/fta-per-fga",
            "Turnovers Per Possession":"stat/turnovers-per-possession",
            "True Shooting %":"stat/true-shooting-percentage",
            "Offensive Rebounding %":"stat/offensive-rebounding-pct",
            "Defensive Rebounding %":"stat/defensive-rebounding-pct",
#             "Total Rebounding %":"stat/total-rebounding-percentage",
            "Block %":"stat/block-pct", 
            "Steal %":"stat/steal-pct",
#             "Assist/Turnover Ratio":"stat/assist--per--turnover-ratio",
            "Defensive Efficiency":"stat/defensive-efficiency",
#             "Effective Possesion Ratio":"stat/effective-possession-ratio",
            "Win %":"stat/win-pct-all-games",
            "Win % Close Games":"stat/win-pct-close-games",
            "Possessions Per Game":"stat/possessions-per-game",
            "Last 5 Rating":"ranking/last-5-games-by-other",
            "Strength of Schedule":"ranking/schedule-strength-by-other" 
            }


In [47]:
def remove_scores(row):
    x = re.sub("\((\d+\-\d+)\)", "", row)
    x = x.rstrip()
    return x

In [48]:
def normalize(df, range):
    min_max_scaler=preprocessing.MinMaxScaler(feature_range=range)
    df = min_max_scaler.fit_transform(df.values)
    return df

In [49]:
def get_stats(years):
    full_data = pd.DataFrame()

    for i in years:
        print("Entering Year: ", i)
        stats=pd.DataFrame()
        suffix = "?date={}-03-01".format(i)
        for key, value in features.items():
            table=pd.DataFrame()
            url = base+value+suffix
            # print('Heading to url: ', url)
            req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
            webpage = urlopen(req).read()
            html = soup(webpage, "html.parser")
            table = pd.read_html(str(html))
            table = table[0].iloc[:, 1:3]
            table = table.set_axis(["Team", key], axis=1, inplace=False)
            if table[key].dtype!='float64':
                table[key] = table[key].replace('--', np.NaN)
                table[key] = table[key].str.rstrip('%').astype(np.float64)
            if key=='Last 5 Rating' or key=='Strength of Schedule':
                table['Team'] = table['Team'].apply(remove_scores)
            if key!='Team' or key!='Year':    
                table[key] = normalize(table[[key]], range=(0, 1))
                # print(table)
                # table[key] = table[key]*table['Strength of Schedule']*table['Last 10 Rating']
                # print(table)
            if key=='Defensive Efficiency':
                table[key]=table[key]*-1
            # table=table.drop(['Last 10 Rating', 'Strength of Schedule'], axis=1)
            if len(stats)==0:
                stats = table
            else:
                # print("entered else statement")
                stats = stats.merge(table, how='inner', on='Team')
        stats['Year'] = i
        full_data = pd.concat([full_data, stats], ignore_index=True)
    return full_data

In [ ]:

x = get_stats(LOOKBACK_PERIOD)

# key_list = ['Year', 'Team']
# drop_list = [w for w in x.columns if w not in key_list]
# print(x[drop_list])
# test = x[drop_list].multiply(x['Strength of Schedule'], axis=0).multiply(x['Last 10 Rating'], axis=0)
# test[key_list] = x[key_list]
# test = test.drop(['Strength of Schedule', 'Last 10 Rating'], axis=1)
# first_column = test.pop('Team')
  
# insert column using insert(position,column_name,first_column) function
# print(test)
# test.insert(0, 'Team', first_column)
# print(test)

Entering Year:  2021


In [43]:
print(x)

                Team  Offensive Efficiency  Effective Field Goal %  \
0             Toledo              1.000000                0.893082   
1            Gonzaga              0.966049                0.993711   
2            Houston              0.910494                0.729560   
3      Youngstown St              0.882716                0.842767   
4       Oral Roberts              0.876543                0.886792   
...              ...                   ...                     ...   
1084     Maryland ES              0.296296                0.201149   
1085     Tarleton St              0.272331                0.281609   
1086      Chicago St              0.257081                0.166667   
1087       Coppin St              0.200436                0.080460   
1088  Miss Valley St              0.000000                0.000000   

      Three Point %  Two Point %  Free Throw %  FT Attempted per FG Attempted  \
0          0.956522     0.730159      0.744292                       0.530405 

In [19]:
# norm_features = ["Floor %","Shooting %", "Effective Field Goal %", "Three Point %", "Two Point %", "Free Throw %", "True Shooting %", "Offensive Rebounding %","Defensive Rebounding %","Total Rebounding %", "Block %"]

# for item in norm_features:
#     x[item] = x[item] / 100

In [20]:
# x["Net Efficiency Margin"] = x["Offensive Efficiency"] - x["Defensive Efficiency"]

KeyError: 'Offensive Efficiency'

In [21]:
x.to_csv("data/{}-{}_MBB_Historical-Data.csv".format(START_YEAR, END_YEAR))

In [22]:
print(x)

                Team  Strength of Schedule  Year
0        Michigan St              1.000000  2012
1             Kansas              0.996183  2012
2            Ohio St              0.980916  2012
3          Wisconsin              0.958015  2012
4            Indiana              0.954198  2012
...              ...                   ...   ...
4589     Maryland ES              0.167742  2025
4590          Wagner              0.148387  2025
4591     Alabama A&M              0.112903  2025
4592   AR-Pine Bluff              0.109677  2025
4593  Miss Valley St              0.000000  2025

[4594 rows x 3 columns]


In [ ]:
clean_hist = pd.read_csv('data/Bracket_Historical_Data.csv')
x = pd.read_csv('data/2012-2025_MBB_Historical-Data.csv')

unknown_teams=[]
for year in x['Year'].unique():
    temp_clean_hist=pd.DataFrame()
    temp_features_df=pd.DataFrame()
    team_list =[]
    temp_clean_hist = clean_hist[clean_hist['Year']==year]
    temp_features_df = x[x['Year']==year]
    team_list = temp_clean_hist[['Team 1', 'Team 2']].to_numpy().flatten()
    team_list = np.unique(team_list)
    for item in team_list:
        if item not in temp_features_df['Team'].values:
            unknown_teams.append(item)
unknown_teams=np.unique(unknown_teams)
print(unknown_teams)



['Abilene Chr' 'American Univ' 'Appalachian St' 'Ark Little Rock'
 'Col Charleston' 'Connecticut' 'Detroit' 'ETSU' 'FL Atlantic'
 'FL Gulf Coast' 'Gardner Webb' 'Hawaii' 'James Madison' 'Kennesaw' 'Kent'
 'LIU Brooklyn' 'Loyola-Chicago' 'MS Valley St' 'MTSU' 'Massachusetts'
 'McNeese St' 'Miami FL' 'North Florida' 'North Texas' 'Northern Iowa'
 'Northwestern LA' 'SUNY Albany' 'South Florida' 'Southern Univ'
 "St Joseph's PA" 'St Louis' "St Mary's CA" "St Peter's" 'TAM C. Christi'
 'TX Southern' 'UC Santa Barbara' 'UNC Asheville' 'UNC Greensboro'
 'UNC Wilmington' 'WI Green Bay' 'WI Milwaukee' 'WKU']
